# Translating a Decision Problem to an Optimization Model

In the first video we discussed a few key concepts that are necessary for mathematical optimization:
- parameters
- decision variables
- constraints
- objective function

In this first modeling example we will see how these are used to formulate a decision problem as an optimization model and code the formulation using `gurobipy`. For more information on all of the commands in the Python API check out our [documentation](https://www.gurobi.com/documentation/10.0/refman/py_python_api_details.html).

## The Decision Problem
We make widgets. Have a set of production facilities that produce boxes of widgets. There is also a set of distribution locations that will then distribute the widgets for sale. Each distribution center has a forecasted demand and each production facility has a min and max number of widgets it can make during this period. We need to ensure that each distribution facility receives enough widgets to satisfy demand from production and we want to do this at minimal cost. The minimum production is 75% of the production facilities max value.

## Sets and Define Model
Our sets are:
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad\quad \texttt{distribution}$

To index each set, we'll use the lowercase letter of each set. Letters used for sets and indices are up to you. Typically, capital letters are for sets and corresponding lowercase will be the index. Single letters are used mainly for conciseness.

In [1]:
%pip install gurobipy

# Import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Sets P and D, respectively
# When we code sets we can be more descriptive in the name
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

# Define a gurobipy model for the decision problem
m = gp.Model('widgets')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.4 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2025-11-24


## Parameters

Parameters of a math optimization problem are values treated as constants in the model and are associated with the decision variables. For this decision problem these values are the limits of each production facility, the demand for each distribution center, and the pairwise costs between production and distribution locations.

- $m_p$ is the max production in location $p$, $\forall p \in P \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{max}\_\texttt{prod[p]}$
- $n_d$ is the number of customers for a distribution center $d$, $\forall d \in D \quad\quad\quad\quad\quad\quad\quad\quad \texttt{n}\_\texttt{demand[d]}$
- $c_{p,d}$ is the cost to ship a widget between location $p$ and location $d$, $\forall p \in P, d \in D \quad\quad\quad \texttt{cost[p,d]}$

In [2]:
# Use squeeze=True to make the costs a series
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1], squeeze=True)
# transp_cost = pd.read_csv('cost.csv', index_col=[0,1], squeeze=True)
# Pivot to view the costs a bit easier
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

<ipython-input-2-7a8fe656262b>:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1], squeeze=True)


distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [3]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
max_prod.to_frame()
#n_demand.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little Rock,140
Birmingham,80
Charleston,180


We also have the requirement that each production facility needs to produce at 75% of this maximum output. We'll denote this value by $a$ in the formulation "frac" for the fraction of maximum production required. Initially we set $a = 0.75$.

In [4]:
frac = 0.75

## Decision Variables
This is what the optimization solver determines, which are the actions you have control over. As a reminder, they come in three main flavors:
- `Continuous`: Price of a product
- `Integer`: The number of food trucks to use for an event
- `Binary`: Yes/no decision to include a certain stock in a portfolio

Decision variables (and parameters) are indexed using elements of sets that we define for the problem. In this example, let's start with a set of cities that produce our widget, which we call set $P$ for the formulation but can define as 'production' in the code. And a set of cities that distribute the widget $D$ and 'distribution' similarly. The decision here is to determine the number of boxes to send from each production facility to each distribution location.

Let $x_{p,d}$ be the number of widgets that are produced at facility $p$ and shipped to location $d$.

### Add Variables in gurobipy
`gurobipy` let's you add decision variables primarily with two (similar) commands:
- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a single variable
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a group of variables by sets/indices

When using `addVars` you have to provide the indices of the variables you want to add, which for us are the production and distribution locations. There are other arguments we can use and will cover a couple of them later on.  

### Our Decision Variables
As is often the case in writing code, there are several ways to get to the same point. Below we can see three different ways to create the decision variables.

In [5]:
# loop through each p and d combination to create a decision variable
m = gp.Model('widgets')
x={}
for p in production:
  for d in distribution:
    x[p,d]=m.addVar(name=p+"_to_"+d)

m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var Baltimore_to_Columbia>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var Baltimore_to_Indianapolis>,
 ('Baltimore', 'Lexington'): <gurobi.Var Baltimore_to_Lexington>,
 ('Baltimore', 'Nashville'): <gurobi.Var Baltimore_to_Nashville>,
 ('Baltimore', 'Richmond'): <gurobi.Var Baltimore_to_Richmond>,
 ('Baltimore', 'St. Louis'): <gurobi.Var Baltimore_to_St. Louis>,
 ('Cleveland', 'Columbia'): <gurobi.Var Cleveland_to_Columbia>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var Cleveland_to_Indianapolis>,
 ('Cleveland', 'Lexington'): <gurobi.Var Cleveland_to_Lexington>,
 ('Cleveland', 'Nashville'): <gurobi.Var Cleveland_to_Nashville>,
 ('Cleveland', 'Richmond'): <gurobi.Var Cleveland_to_Richmond>,
 ('Cleveland', 'St. Louis'): <gurobi.Var Cleveland_to_St. Louis>,
 ('Little Rock', 'Columbia'): <gurobi.Var Little Rock_to_Columbia>,
 ('Little Rock', 'Indianapolis'): <gurobi.Var Little Rock_to_Indianapolis>,
 ('Little Rock', 'Lexington'): <gurobi.Var Little Rock_to_Le

In [ ]:
# Provide each set for the indices
m = gp.Model('widgets')


In [6]:
# The index of the tranporation costs have each combination of prodiction and distribution location
m = gp.Model('widgets')
x=m.addVars(transp_cost.index,name="prod_ship")
m.update()
x


{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

The command `m.update()` updates the model to include any changes that have been made, like adding variables. It doesn't need to be run in every cell but if you see *Awaiting Model Update* in the output of a cell, then this should prevent that from happening.

## Constraints
We outlined production and demand constraints at the beginning of this example; now we formulate and code them. Note that it doesn't matter the order in which constraints (and/or decision variables) are added to the model.

### Add Constraints in gurobipy
Adding constraints to a model is similar to adding variables:
- [addConstr()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstr.html) adds a single constraint
- [addConstrs()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstrs.htmll) adds a group of constraints using a Python `generator` expression

### Our Constraints
To start, we'll formulate the demand constraints for each distribution location first and add them to the model.

\begin{align*}
\sum_{p}x_{p,d} \ge n_d, \quad \forall d \in D \quad\quad \texttt{meet}\_\texttt{demand[d]}\\
\end{align*}

This will be the first time we use [gp.quicksum()](https://www.gurobi.com/documentation/10.0/refman/py_quicksum.html). There are other ways to sum expressions in gurobipy and while this method isn't the most concise to code, it is easy to compare it to the summation in the formulation to see how it works.

In [7]:
meet_demand=m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution),
                          name="meet_demand")
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

Next we have the maximum number of widgets each production facility can make. We also have that each facility must make at least 75% of its max production.

$$
\begin{align*}
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P \quad\quad &\texttt{can}\_\texttt{produce[p]}\\
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P \quad\quad &\texttt{must}\_\texttt{produce[p]}\\
\end{align*}
$$


In [8]:
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution)<= max_prod[p] for p in production),
                           name="can produce")
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution)>= frac*max_prod[p] for p in production),
                           name="must produce")
m.update()
can_produce

{'Baltimore': <gurobi.Constr can produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can produce[Cleveland]>,
 'Little Rock': <gurobi.Constr can produce[Little Rock]>,
 'Birmingham': <gurobi.Constr can produce[Birmingham]>,
 'Charleston': <gurobi.Constr can produce[Charleston]>}

## Objective Function
We were told to **reduce** the transportation costs and we'll use this to determine our objective function as minimizing the total cost to ship widgets from production to distribution locations.

### Setting the Objective in gurobipy
This is done using [setObjective()](https://www.gurobi.com/documentation/10.0/refman/py_model_setobjective.html). The second argument (in this case `GRB.MINIMIZE`) is called the model's *sense*. For a maximization problem we would use `GRB.MAXIMIZE`.

### Our Objective Function
\begin{align*}
{\rm minimize} \space \sum_{p,d}c_{p,d}x_{p,d}, \quad \forall p \in P, d \in D\\
\end{align*}

In [9]:
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

## Find, Extract, and Analyze the Solution
Before running the optimization, it is a good idea to write an `lp` file. This is a text file that prints out the variables, constraints, and object like we would see in the *formulation*, just without the summation symbols and using the names we designated.

In [10]:
m.write('widget_shipment.lp')

### Run the Optimization

In [11]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.704890000e+03


### Extract the Solution
There are many ways to get the values of decision variables out of gurobipy.

In [12]:
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
#sol
sol[sol.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

Here are a couple of other ways to get the solution.

In [13]:
# You can get the name and value of all the decision variables:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'prod_ship[Baltimore,Columbia]': 0.0,
 'prod_ship[Baltimore,Indianapolis]': 0.0,
 'prod_ship[Baltimore,Lexington]': 0.0,
 'prod_ship[Baltimore,Nashville]': 19.0,
 'prod_ship[Baltimore,Richmond]': 116.0,
 'prod_ship[Baltimore,St. Louis]': 0.0,
 'prod_ship[Cleveland,Columbia]': 89.0,
 'prod_ship[Cleveland,Indianapolis]': 95.0,
 'prod_ship[Cleveland,Lexington]': 0.0,
 'prod_ship[Cleveland,Nashville]': 2.0,
 'prod_ship[Cleveland,Richmond]': 0.0,
 'prod_ship[Cleveland,St. Louis]': 0.0,
 'prod_ship[Little Rock,Columbia]': 0.0,
 'prod_ship[Little Rock,Indianapolis]': 0.0,
 'prod_ship[Little Rock,Lexington]': 0.0,
 'prod_ship[Little Rock,Nashville]': 0.0,
 'prod_ship[Little Rock,Richmond]': 0.0,
 'prod_ship[Little Rock,St. Louis]': 140.0,
 'prod_ship[Birmingham,Columbia]': 0.0,
 'prod_ship[Birmingham,Indianapolis]': 0.0,
 'prod_ship[Birmingham,Lexington]': 0.0,
 'prod_ship[Birmingham,Nashville]': 80.0,
 'prod_ship[Birmingham,Richmond]': 0.0,
 'prod_ship[Birmingham,St. Louis]': 0.0,
 'prod_shi

Or you can only iterate over a specific variable and only return values that are of interest to you. Remember, x is a dict in python. So, iterate over it, the same way you iterate over any dictionary

In [14]:
xvals = {k: v.x for k,v in x.items() if v.x > 0}
xvals

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little Rock', 'St. Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St. Louis'): 41.0}

### Solution Analysis
While determining the optimal transportation of widgets was our goal, we may want to dig a little deeper into the solution. For example we can aggregate the total production by facility to see which locations (if any) did not produce their maximum capacity of widgets and which (if any) production facilities are at the lower bound of their production.

In [15]:
# Sum the shipment amount by production facility
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

,Remaining,Utilization
Baltimore,45.0,0.75
Birmingham,0.0,1.00
Charleston,18.0,0.90
Cleveland,14.0,0.93
Little Rock,0.0,1.00


In mathematical optimization, when the left-hand and right-hand sides of an inequality constraint are equal, we say the constraint is `binding`. When this *doesn't happen* then there is `slack` or `surplus` in that constraint. We can get this value by calling the `Slack` attribute of a constraint.

In [16]:
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production],
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]},
             index = production)

,Remaining,Utilization
Baltimore,45.0,0.75
Cleveland,14.0,0.93
Little Rock,0.0,1.00
Birmingham,0.0,1.00
Charleston,18.0,0.90


# Using Binary Variables
As we described in the first session and also at the top of this notebook, binary variables are used to choose alternatives in mathematical optimization. They can be interpreted as a yes/no decision or an on/off switch.

In the original problem Birmingham's production is much lower than the rest of the facilities. Suppose we have the option to expand that facilities max capacity by either 25 or 50 widgets, but there is a cost of \\$50 and \\$75, respectively, to choose one of these options and we can choose at most one. We'll use a binary decision variable for each option named $xprod$.

Let $xprod_0 = 1$ if we choose the first option and expand production capacity by 25 and $0$ otherwise.
Let $xprod_1 = 1$ if we choose the second option and expand production capacity by 50 and $0$ otherwise.

While it's fairly common to use single lowercase letters as decision variables, it is not necessary and you'll see variables defined as above (where they are more descriptive) quite often. We will formulate a new model that contains the same decision variables and demand constraints as before.

In [17]:
# We use m2 for the second model
# These parts are the same as above outside of the new model name
m2 = gp.Model('widgets2')
x = m2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
meet_demand = m2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

In the cell above we did use new argument of the `addVars()` function: `obj`. This will set the coefficient of the added decision variables in the objective function and is equivalent to what we did earlier by attaching the transportation costs between each production and distribution location to the appropriate decision variable.  

Next, we'll add the same constraints for production limits as before for each production facility other than Birmingham. The formulation is basically the same other than the set the constraints hold for.
$$
\begin{align*}
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P -\{\texttt{Birmingham}\} \\
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P -\{\texttt{Birmingham}\} \\
\end{align*}
$$

In gurobipy, this is done by adding a condition in the `generator` expression.

Now, add the new binary variables.

Let's breakdown each of the arguments in the cell above -- there are a few new things there.
1. `range(n)` is used to add $n$ decision variables. In this case we add 2 variables.
2. We need to declare this as a binary variable using `vtype`.
3. We again use the `obj` capability to immediately set the objective function coefficients for these variables.

The objective and new binary variables look like this in the formulation:

\begin{align*}
{\rm minimize} \space &\sum_{p,d}c_{p,d}x_{p,d} + 50*xprod_0 + 75*xprod_1, \quad &\forall p \in P, d \in D\\
&xprod_i \in \{0,1\}, &{\rm for} \space i \in \{0,1\}
\end{align*}

Next we have the production constraints that are specific to the Birmingham facility.

$$
\begin{align*}
\sum_{d}x_{p,d} &\le m_p + 25*xprod_0 + 50*xprod_1, & p = \texttt{Birmingham} \\
\sum_{d}x_{p,d} &\ge a*(m_p+ 25*xprod_0 + 50*xprod_1),& p = \texttt{Birmingham} \\
\end{align*}
$$

It was stated above that we can select at most one of the expansion options which means we cannot allow both $xprod_0$ and $xprod_1$ to equal one. To model this we add a constraint limiting the sum of these two binary variables to at most one.

$$
\begin{align*}
\sum_{i}xprod_i \le 1
\end{align*}
$$
The corresponding constraint in gurobipy:

Now we can run this optimization model and see if this potential expansion will help us reduce overall costs.

In [ ]:
m2.optimize()

In [ ]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"The original model had a total cost of {round(obj1.getValue(),2)}")
print(f"The new formualtion has a total cost of {round(obj2.getValue(),2)}")

What does the change in objective function value tell us?

Let's look at values of our binary variables.

In [ ]:
pd.Series(m2.getAttr('X', xprod))

The model selected the first expansion option since $xprod_0 = 1$, which was increasing production by 25 widgets in Birmingham. We can see the rest of the solution, which will include the increase in Birmingham's production capacity.

In [ ]:
x2_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

### Homework! (not really, but something to look into)
Analyze how the optimal solution changes between the two models. You'll notice something weird.
- What is it that's odd?
- Why do you think it happened?
- How would you address it from formulation perspective and a business perspective?